In [92]:
from collections import defaultdict
import math 

def beam_search(words,possible_tags,P_T,P_E):
    B=3
    l=len(words)
    best_score={"0 <s>":0}
    best_edge={"0 <s>":None}
    active_tags=defaultdict(lambda:0)
    active_tags[0]=["<s>"]
    lam=0.05
    N=10**5
    for i in range(l):
        my_best={}
        for prev in active_tags[i]:
            for next_ in possible_tags:
                if f"{i} {prev}" in best_score and f"T {prev} {next_}" in P_T:
                    score=best_score[f"{i} {prev}"]-math.log(P_T[f"T {prev} {next_}"],2)-math.log(lam*P_E[f"E {next_} {words[i]}"]+(1-lam)/N,2)
                    if f"{i+1} {next_}" not in best_score or best_score[f"{i+1} {next_}"]>score:
                        best_score[f"{i+1} {next_}"]=score
                        best_edge[f"{i+1} {next_}"]=f"{i} {prev}"
                        my_best[next_]=score
        sort_tags=[k for k in sorted(my_best, key=my_best.get)]
        active_tags[i+1]=sort_tags[:B]
       
    for tag in possible_tags:
        if f"T {tag} <\s>" in P_T and f"{l} {tag}" in best_score:
            score=best_score[f"{l} {tag}"]-math.log(P_T[f"T {tag} <\s>"])
            if f"{l+1} <\s>" not in best_score or best_score[f"{l+1} <\s>"]>score:
                best_score[f"{l+1} <\s>"]=score
                best_edge[f"{l+1} <\s>"]=f"{l} {tag}"
            
    tag_path=[]
    next_edge=best_edge[f"{l+1} <\s>"]
    while next_edge!="0 <s>":
        no,tag=next_edge.split()
        tag_path.append(tag)
        next_edge=best_edge[next_edge]
    tag_path.reverse()
    return tag_path


with open("wiki-en-train.norm_pos.txt","r",encoding="utf-8")as f:
    emit=defaultdict(int)
    transition=defaultdict(int)
    context=defaultdict(int)
    for line in f:
        previous="<s>"
        context[previous]+=1
        wordtags=line.strip().split()
        for wordtag in wordtags:
            word,tag=wordtag.split("_")
            transition[f"{previous} {tag}"]+=1
            context[f"{tag}"]+=1
            emit[f"{tag} {word}"]+=1
            previous=tag
        transition[f"{previous} <\s>"]+=1
    P_T=defaultdict(float)
    P_E=defaultdict(float)
    possible_tags=defaultdict(int)
    for key,value in transition.items():
        tag1,tag2=key.split()
        possible_tags[tag1]+=1
    
    for key,value in transition.items():
        previous,word=key.split(" ")
        prob=value/context[previous]
        P_T[f"T {key}"]=prob

    for key,value in emit.items():
        previous,word=key.split(" ")
        prob=value/context[previous]
        P_E[f"E {key}"]=prob
    
        
        
 
    with open("wiki-en-test.norm.txt","r",encoding="utf-8") as f1:
        for line in f1:
            X=line.strip().split()
            tag_path=beam_search(X,possible_tags,P_T,P_E)
            with open("answer.txt","a",encoding="utf-8") as ans:
                ans.write(" ".join(tag_path)+"\n")
    


In [93]:
with open("answer.txt","r",encoding="utf-8") as ans:
    Y_test=[]
    for line in ans:
        tags=line.strip().split()
        Y_test.append(tags)

with open("wiki-en-test.norm_pos.txt","r",encoding="utf-8") as f2:
    Y_ans=[]
    for line in f2:
        word_pos=line.strip().split()
        Y=[]
        for w_p in word_pos:
            x,y=w_p.split("_")
            Y.append(y)
        Y_ans.append(Y)
    
    count=0
    correct=0
    for test,ans in zip(Y_test,Y_ans):
        for i,j in zip(test,ans):
            count+=1
            if(i==j):
                correct+=1
    print(str(correct/count))

0.873767258382643
